In [25]:
from caveclient import CAVEclient, auth
from caveclient import chunkedgraph as cg
import gcsfs
from google.cloud import storage
import pandas as pd
import numpy as np
import time
import json

In [26]:
url = "https://pcgv3local.brain-wire-test.org"
global_url = "https://global.brain-wire-test.org/"
datastack = "fish1_full"
dataset = "fish_v241003_test"
resolution = [16, 16, 30]

In [27]:
client = CAVEclient(datastack_name=datastack, server_address=global_url)
cggraph = cg.ChunkedGraphClient(server_address=url, table_name=dataset, auth_client=auth.AuthClient(token=client.auth.token))
cv = client.info.segmentation_cloudvolume()

Updated datastack-to-server cache — 'https://global.brain-wire-test.org/' will now be used by default for datastack 'fish1_full'


In [28]:
from pychunkedgraph.graph import ChunkedGraph

In [29]:
pcg = ChunkedGraph(graph_id=dataset)

In [30]:
pcg.client.get_max_operation_id()

4980

In [6]:
key_path = "/root/.cloudvolume/secrets/fish1-pychunkedgraph-google-secret.json"
project_id = "engert-goog-connectomics"
bucket_name = 'fish1-pychunkedgraph'
folder_name = 'fish1-long-edges-merge-v241003'
dest_bucket_name = "fish1-long-edges-merge-v241003-errors" # replace with your destination bucket name


gsutil du gs://fish1-pychunkedgraph/fish1-long-edges-merge-v241003 | wc -l

In [7]:
from google.cloud import storage
from tqdm import tqdm

def get_bucket_file_names():
    client = storage.Client.from_service_account_json(key_path)
    bucket = client.bucket(bucket_name)
    blobs = client.list_blobs(bucket, prefix=folder_name)
    
    for blob in tqdm(blobs, desc="Processing blobs"):
        if "_False.json" in blob.name:
            yield blob


In [32]:
def get_latest_root_id(x, y, z):
    supervoxel_id = cv.download_point(pt=(x, y, z), 
                                      size=1, 
                                      agglomerate=False, 
                                      coord_resolution=resolution)
    supervoxel_id = np.int64(supervoxel_id[0, 0, 0, 0]) 
    root_id = cggraph.get_root_id(supervoxel_id)
    # print(root_id, type(root_id))
    # latest_root_id = cggraph.get_latest_roots(root_id)
    # print(latest_root_id, type(latest_root_id))
    
    return root_id, supervoxel_id

In [31]:
import json
from caveclient.base import BaseEncoder, handle_response

def custom_do_merge(supervoxels, coords, resolution=(4, 4, 40), params={
        "priority": False,
        "chebyshev_distance": 10
    }) -> None:
    
    endpoint_mapping = cggraph.default_url_mapping
    url = cggraph._endpoints["do_merge"].format_map(endpoint_mapping)

    data = []
    for svid, coor in zip(supervoxels, coords):
        pos = np.array(coor) * resolution
        row = [svid, pos[0], pos[1], pos[2]]
        data.append(row)
    params = params
    response = cggraph.session.post(
        url,
        data=json.dumps(data, cls=BaseEncoder),
        params=params,
        headers={"Content-Type": "application/json"},
    )
    handle_response(response)

In [8]:
file_paths = get_bucket_file_names()

In [6]:
len(file_paths)

0

In [34]:
data = {
  "did_merge": False,
  "same_root_id": False,
  "seg1_target_graph_sv_id": 87613968067068050,
  "seg2_target_graph_sv_id": 93665269360100690,
  "seg1_sv_id": 15960395825,
  "seg2_sv_id": 26734455752,
  "seg1_sv_id_loc": [56667, 18263, 4820],
  "seg2_sv_id_loc": [78659, 16884, 7831],
  "seg1_root": 864691128684458000,
  "seg2_root": 864691128583231900
}

In [49]:
data = {
    "seg1_root": 864691128670960582,
    "seg1_target_graph_sv_id": 78608004957604489, 
    "seg1_sv_id": 11141422389, 
    "seg2_root": 864691128580106831,
    "seg2_target_graph_sv_id": 78678236329935211,
    "seg2_sv_id": 11182284222,
    "seg1_sv_id_loc": [23824, 23011, 3284],
    "seg2_sv_id_loc": [24263, 22418, 3334]
}

In [51]:
seg1_id = data['seg1_sv_id'] 
seg2_id = data['seg2_sv_id']
seg1_x, seg1_y, seg1_z = data['seg1_sv_id_loc']
seg2_x, seg2_y, seg2_z = data['seg2_sv_id_loc']
seg1_root_id, seg1_supervoxel_id = get_latest_root_id(seg1_x, seg1_y, seg1_z)
seg2_root_id, seg2_supervoxel_id = get_latest_root_id(seg2_x, seg2_y, seg2_z)
print(seg1_root_id, seg2_root_id)
print(seg1_supervoxel_id, seg2_supervoxel_id)


Decompressing: 100%|██████████| 1/1 [00:00<00:00, 1498.50it/s]

864691128663162995 864691128580106831
78608004957604489 78678236329935211


In [47]:
seg1_id = data['seg1_sv_id'] 
seg2_id = data['seg2_sv_id']
seg1_x, seg1_y, seg1_z = data['seg1_sv_id_loc']
seg2_x, seg2_y, seg2_z = data['seg2_sv_id_loc']
seg1_root_id, seg1_supervoxel_id = get_latest_root_id(seg1_x, seg1_y, seg1_z)
seg2_root_id, seg2_supervoxel_id = get_latest_root_id(seg2_x, seg2_y, seg2_z)
print(seg1_root_id, seg2_root_id)
print(seg1_supervoxel_id, seg2_supervoxel_id)


Decompressing: 100%|██████████| 1/1 [00:00<00:00, 1579.78it/s]

864691128680989822 864691128680989822
87613968067068048 93665269360100686


In [52]:
pcg.client.get_max_operation_id()

4983

In [54]:
cggraph.get_operation_details([4980,4981])

{'4980': {'added_edges': [[81842698171973649, 81912036459545846]],
  'operation_exception': '',
  'operation_status': 0,
  'operation_ts': '2024-12-19 03:35:59.107000+00:00',
  'roots': [864691128676176554],
  'sink_coords': [[35958, 10564, 1533]],
  'source_coords': [[35595, 14385, 832]],
  'timestamp': '2024-12-19 03:35:59.107000+00:00',
  'user': '31'},
 '4981': {'added_edges': [[87613968067068048, 93665269360100686]],
  'operation_exception': '',
  'operation_status': 0,
  'operation_ts': '2025-01-07 13:50:30.866000+00:00',
  'roots': [864691128680989822],
  'sink_coords': [[78659, 16884, 7831]],
  'source_coords': [[56667, 18263, 4820]],
  'timestamp': '2025-01-07 13:50:30.866000+00:00',
  'user': '31'}}

In [53]:
cggraph.get_operation_details([4982, 4983])

{'4982': {'added_edges': [[87613968067068048, 93665269360100686]],
  'operation_exception': '',
  'operation_status': 1,
  'operation_ts': '2025-01-07 13:56:49.329000+00:00',
  'roots': [],
  'sink_coords': [[78659, 16884, 7831]],
  'source_coords': [[56667, 18263, 4820]],
  'timestamp': '2025-01-07 13:56:49.329000+00:00',
  'user': '31'},
 '4983': {'added_edges': [[87613968067068048, 93665269360100686]],
  'operation_exception': '',
  'operation_status': 1,
  'operation_ts': '2025-01-07 13:58:10.862000+00:00',
  'roots': [],
  'sink_coords': [[78659, 16884, 7831]],
  'source_coords': [[56667, 18263, 4820]],
  'timestamp': '2025-01-07 13:58:10.862000+00:00',
  'user': '31'}}